In [1]:
#importing necessary libraries
import pandas as pd
import string
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import string
import re
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.naive_bayes import GaussianNB

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#loading the dataset after converting tsv into csv
train_data = pd.read_csv("/content/drive/MyDrive/olid-training.csv")
train_data

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
...,...,...,...,...,...
13188,94475,@USER every antifa member is just several midg...,NOT,NaN,NaN
13189,34150,@USER Berkeley Antifa do not agree with you URL,NOT,NaN,NaN
13190,83400,@USER You are a great model for inspiration,NOT,NaN,NaN
13191,84081,"@USER Chris, Chris, Chris.... Are you forgetti...",OFF,UNT,NaN


In [5]:
#displaying the tweets
train_tweets = train_data[["tweet"]]
print(train_tweets)

                                                   tweet
0      @USER She should ask a few native Americans wh...
1      @USER @USER Go home you’re drunk!!! @USER #MAG...
2      Amazon is investigating Chinese employees who ...
3      @USER Someone should'veTaken" this piece of sh...
4      @USER @USER Obama wanted liberals &amp; illega...
...                                                  ...
13188  @USER every antifa member is just several midg...
13189    @USER Berkeley Antifa do not agree with you URL
13190        @USER You are a great model for inspiration
13191  @USER Chris, Chris, Chris.... Are you forgetti...
13192  @USER Advocate for gun control while breaking ...

[13193 rows x 1 columns]


In [6]:
train_data

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
...,...,...,...,...,...
13188,94475,@USER every antifa member is just several midg...,NOT,NaN,NaN
13189,34150,@USER Berkeley Antifa do not agree with you URL,NOT,NaN,NaN
13190,83400,@USER You are a great model for inspiration,NOT,NaN,NaN
13191,84081,"@USER Chris, Chris, Chris.... Are you forgetti...",OFF,UNT,NaN


In [7]:
import string
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

def process_tweets(df, ngram_range=(1, 3), stop_words='english'):
    # Function to clean tweets
    def clean_tweet(tweet):
        punctuations = string.punctuation
        stop_words = set(stopwords.words('english'))

        # Remove mentions, URLs, and special characters
        tweet = tweet.replace('@USER', '') \
                     .replace('URL', '') \
                     .replace('&amp', 'and') \
                     .replace('&lt','') \
                     .replace('&gt','') \
                     .replace('\d+','') \
                     .lower()

        # Remove punctuations
        for punctuation in punctuations:
            tweet = tweet.replace(punctuation, '')

        # Remove emojis
        tweet = re.sub(r'[^\x00-\x7F]+', '', tweet)

        # Remove stopwords
        tweet = ' '.join([word for word in tweet.split() if word not in stop_words])

        # Trim leading and trailing whitespaces
        tweet = tweet.strip()

        return tweet

    # Function to lemmatize tweets
    def lemmatize_tweet(tokens):
        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
        return lemmatized_tokens

    # Clean tweets
    df['cleaned_tweet'] = df['tweet'].apply(clean_tweet)

    # Tokenize tweets
    df['tokenized_tweet'] = df['cleaned_tweet'].apply(word_tokenize)

    # Lemmatize tweets
    df['lemmatized_tweet'] = df['tokenized_tweet'].apply(lemmatize_tweet)

    # Join lemmatized tokens back into strings
    df['lemmatized_tweet'] = df['lemmatized_tweet'].apply(lambda x: ' '.join(x))

    # Vectorize tweets using TF-IDF
    tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=ngram_range)
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['lemmatized_tweet'])

    return tfidf_matrix, tfidf_vectorizer.get_feature_names_out()

# Example usage:
# Assuming train_tweets is a DataFrame with a column named 'tweet'
tfidf_matrix, feature_names = process_tweets(train_tweets)
print("TF-IDF Matrix Shape:", tfidf_matrix.shape)
print("Feature Names:", feature_names)


<ipython-input-7-4a6b54e8f354>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_tweet'] = df['tweet'].apply(clean_tweet)
<ipython-input-7-4a6b54e8f354>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokenized_tweet'] = df['cleaned_tweet'].apply(word_tokenize)
<ipython-input-7-4a6b54e8f354>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

TF-IDF Matrix Shape: (13193, 203202)
Feature Names: ['00000' '00000 fb' '005' ... 'zuckerberg' 'zuckerberg soros'
 'zuckerberg soros xi']


In [8]:
process_tweets(train_data, ngram_range=(1, 3), stop_words='english')

(<13193x203202 sparse matrix of type '<class 'numpy.float64'>'
 	with 322265 stored elements in Compressed Sparse Row format>,
 array(['00000', '00000 fb', '005', ..., 'zuckerberg', 'zuckerberg soros',
        'zuckerberg soros xi'], dtype=object))

In [9]:
train_data

,id,tweet,subtask_a,subtask_b,subtask_c,cleaned_tweet,tokenized_tweet,lemmatized_tweet
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,ask native americans take,"[ask, native, americans, take]",ask native american take
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,go home youre drunk maga trump2020,"[go, home, youre, drunk, maga, trump2020]",go home youre drunk maga trump2020
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,amazon investigating chinese employees selling...,"[amazon, investigating, chinese, employees, se...",amazon investigating chinese employee selling ...
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,someone shouldvetaken piece shit volcano,"[someone, shouldvetaken, piece, shit, volcano]",someone shouldvetaken piece shit volcano
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,obama wanted liberals illegals move red states,"[obama, wanted, liberals, illegals, move, red,...",obama wanted liberal illegals move red state
...,...,...,...,...,...,...,...,...
13188,94475,@USER every antifa member is just several midg...,NOT,NaN,NaN,every antifa member several midget gypsies sta...,"[every, antifa, member, several, midget, gypsi...",every antifa member several midget gypsy stand...
13189,34150,@USER Berkeley Antifa do not agree with you URL,NOT,NaN,NaN,berkeley antifa agree,"[berkeley, antifa, agree]",berkeley antifa agree
13190,83400,@USER You are a great model for inspiration,NOT,NaN,NaN,great model inspiration,"[great, model, inspiration]",great model inspiration
13191,84081,"@USER Chris, Chris, Chris.... Are you forgetti...",OFF,UNT,NaN,chris chris chris forgetting mantra every sing...,"[chris, chris, chris, forgetting, mantra, ever...",chris chris chris forgetting mantra every sing...


In [10]:
df = train_data[["lemmatized_tweet", "subtask_a", "subtask_b", "subtask_c"]]
df

,lemmatized_tweet,subtask_a,subtask_b,subtask_c
0,ask native american take,OFF,UNT,NaN
1,go home youre drunk maga trump2020,OFF,TIN,IND
2,amazon investigating chinese employee selling ...,NOT,NaN,NaN
3,someone shouldvetaken piece shit volcano,OFF,UNT,NaN
4,obama wanted liberal illegals move red state,NOT,NaN,NaN
...,...,...,...,...
13188,every antifa member several midget gypsy stand...,NOT,NaN,NaN
13189,berkeley antifa agree,NOT,NaN,NaN
13190,great model inspiration,NOT,NaN,NaN
13191,chris chris chris forgetting mantra every sing...,OFF,UNT,NaN


In [11]:
#mapping data

In [12]:
df['subtask_a'] = df['subtask_a'].map({'OFF': 1, 'NOT': 0})
df['subtask_b'] = df['subtask_b'].map({'TIN': 1, 'UNT': 0})
df['subtask_c'] = df['subtask_c'].map({'OTH':2, 'IND': 1, 'GRP': 0})


<ipython-input-12-e772aeaa11c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subtask_a'] = df['subtask_a'].map({'OFF': 1, 'NOT': 0})
<ipython-input-12-e772aeaa11c5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subtask_b'] = df['subtask_b'].map({'TIN': 1, 'UNT': 0})
<ipython-input-12-e772aeaa11c5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [13]:
X_subtaska= df['lemmatized_tweet']
y_subtaska= df['subtask_a']

SUBTASK A (OFFENSIVE OR NOT OFFENSIVE)

In [14]:
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler

# Assuming X_subtaska contains preprocessed and lemmatized text data and y_subtaska contains corresponding labels

# Perform downsampling to balance the classes
rus = RandomOverSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_subtaska.values.reshape(-1, 1), y_subtaska)


In [15]:
X_resampled

array([['ask native american take'],
       ['go home youre drunk maga trump2020'],
       ['amazon investigating chinese employee selling internal data thirdparty seller looking edge competitive marketplace amazon maga kag china tcot'],
       ...,
       ['yeahhhhh admit destiny 1 cayde liker god made suck 2'],
       ['exactly havent watched two year refuse former fan think jerry jones disgrace people helped make rich'],
       ['censored qanon merchandise really zazzle meanwhile communism antifa merch allowed site constitution usa mean nothing well done time take merch elsewhere wwg1wga']],
      dtype=object)

In [16]:
# Vectorize the resampled text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features = 100)
X_resampled_tfidf = tfidf_vectorizer.fit_transform(X_resampled.ravel())

In [17]:
# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_resampled_tfidf, y_resampled, test_size=0.2, random_state=42)


In [18]:
# Initialize and train the RandomForestClassifier model
model_a = RandomForestClassifier(n_estimators=300, min_samples_leaf = 10, min_samples_split= 10, max_depth = 10)
model_a.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=10, min_samples_split=10,
                       n_estimators=300)

In [19]:

# Evaluate your model
y_train_pred = model_a.predict(X_train)
print("Training Classification Report:")
print(classification_report(y_train, y_train_pred))

y_valid_pred = model_a.predict(X_valid)
print("Validation Classification Report:")
print(classification_report(y_valid, y_valid_pred))


Training Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.88      0.71      7039
           1       0.77      0.40      0.52      7047

    accuracy                           0.64     14086
   macro avg       0.68      0.64      0.62     14086
weighted avg       0.68      0.64      0.62     14086

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.85      0.69      1765
           1       0.72      0.39      0.50      1757

    accuracy                           0.62      3522
   macro avg       0.65      0.62      0.60      3522
weighted avg       0.65      0.62      0.60      3522



SUBTASK B (Targeted or Untargeted)

In [20]:
#filtering data for passing to model 2
X_subtaskb = X_subtaska[df['subtask_a'] == 1]
y_subtaskb = df.loc[df['subtask_a'] == 1, 'subtask_b']


In [21]:
X_subtaskb

0                                 ask native american take
1                       go home youre drunk maga trump2020
3                 someone shouldvetaken piece shit volcano
5                                           liberal kookoo
6                                         oh no tough shit
                               ...                        
13183                    thats shaq silver back looking as
13184    cant dense point cant presume someone guilty t...
13185    google cfo joke idiot hired position googleare...
13187                                  human garbage wrong
13191    chris chris chris forgetting mantra every sing...
Name: lemmatized_tweet, Length: 4389, dtype: object

In [22]:
y_subtaskb

0        0.0
1        1.0
3        0.0
5        1.0
6        0.0
        ... 
13183    0.0
13184    1.0
13185    1.0
13187    1.0
13191    0.0
Name: subtask_b, Length: 4389, dtype: float64

In [23]:
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler

# Assuming X_subtaska contains preprocessed and lemmatized text data and y_subtaska contains corresponding labels

# Perform downsampling to balance the classes
rus = RandomOverSampler(random_state=42)
X_resampled_b, y_resampled_b = rus.fit_resample(X_subtaskb.values.reshape(-1, 1), y_subtaskb)


In [24]:
len(X_resampled_b)

7736

In [25]:
# Vectorize the resampled text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features = 100)
X_resampled_tfidf_b = tfidf_vectorizer.fit_transform(X_resampled_b.ravel())

In [26]:
# Split the data into training and validation sets
X_train_b, X_valid_b, y_train_b, y_valid_b = train_test_split(X_resampled_tfidf_b, y_resampled_b, test_size=0.2, random_state=42)


In [37]:
# Initialize and train the RandomForestClassifier model
if X_resampled_tfidf_b.shape[0] > 0:
  model_b = RandomForestClassifier()
  model_b.fit(X_train_b, y_train_b)
  # Evaluate your model
  y_train_pred_b = model_b.predict(X_train_b)
  print("Training Classification Report:")
  print(classification_report(y_train_b, y_train_pred_b))
  y_valid_pred_b = model_b.predict(X_valid_b)
  print("Validation Classification Report:")
  print(classification_report(y_valid_b, y_valid_pred_b))

else:
  y_train_pred_b = []

Training Classification Report:
              precision    recall  f1-score   support

         0.0       0.83      0.97      0.89      3090
         1.0       0.96      0.80      0.87      3098

    accuracy                           0.88      6188
   macro avg       0.89      0.88      0.88      6188
weighted avg       0.89      0.88      0.88      6188

Validation Classification Report:
              precision    recall  f1-score   support

         0.0       0.80      0.97      0.87       778
         1.0       0.96      0.75      0.84       770

    accuracy                           0.86      1548
   macro avg       0.88      0.86      0.86      1548
weighted avg       0.88      0.86      0.86      1548



In [38]:
test_data_b = pd.read_csv("/content/drive/MyDrive/testset-levelb.tsv", delimiter='\t')

In [39]:
#preprocessing test data
preprocessed_test_data = process_tweets(test_data_b, ngram_range=(1, 3), stop_words='english')

In [40]:
X_test_tfidf_b = tfidf_vectorizer.fit_transform(test_data_b["lemmatized_tweet"].ravel())

In [41]:
predictions_b = model_b.predict(X_test_tfidf_b)

In [42]:
test_labels_b = pd.read_csv("/content/labels-levelb.csv")

In [43]:
test_labels_b["labels_b"] = test_labels_b["labels_b"].map({'TIN': 1, 'UNT': 0})

In [44]:
test_labels_b

,id,labels_b
0,15923,1
1,60133,1
2,83681,1
3,65507,1
4,12588,0
...,...,...
235,22569,1
236,48938,1
237,41438,1
238,73439,1


In [45]:
print(classification_report(test_labels_b["labels_b"], predictions_b))

              precision    recall  f1-score   support

           0       0.13      0.22      0.16        27
           1       0.89      0.81      0.85       213

    accuracy                           0.75       240
   macro avg       0.51      0.52      0.51       240
weighted avg       0.81      0.75      0.77       240



In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/testset-levela.tsv", delimiter='\t')

In [ ]:
#preprocessing test data
preprocessed_test_data1 = process_tweets(test_data, ngram_range=(1, 3), stop_words='english')

In [ ]:
preprocessed_test_data1

In [ ]:
test_data

In [ ]:
X_test_tfidf = tfidf_vectorizer.fit_transform(test_data["lemmatized_tweet"].ravel())

In [ ]:
predictions_a = model_a.predict(X_test_tfidf)

In [ ]:
#offensive : 1, not_offensive : 0
predictions_a

In [ ]:
test_labels = pd.read_csv("/content/labels-levela.csv")

In [ ]:

test_labels

In [ ]:
test_labels.info()

In [ ]:
test_labels["label"] = test_labels["label"].map({'OFF': 1, 'NOT': 0})

In [ ]:
print(classification_report(test_labels["label"], predictions_a))

In [ ]:

custom_tweets = pd.read_csv("/content/custom_tweet.csv")

In [ ]:
custom_tweets

In [36]:
custom_df = custom_tweets[["tweet", "labels_self"]]

In [37]:
custom_df

,tweet,labels_self
0,"Blyatnsky Army- “Run the fuck, why are you so ...",OFF
1,“you dont get to tell me about sad” well fuck,OFF
2,Chelsea’s insecurities drove her to put hersel...,OFF
3,Nah all these islanders are pussies they need ...,OFF
4,"Hello, I’m 6’5 and I have a private jet. I can...",NOT
5,Guys are sending their n*des to me in the dms ...,OFF
6,"Met a fabulous entrepreneur for lunch, Rachana...",NOT
7,A bicycle race on ocean with different animals...,NOT
8,Only hot people can complete the whole syllabu...,NOT
9,"You don't need to apologise @USER bhai, these ...",NOT


In [38]:
#preprocessing custom data
preprocessed_test_data1 = process_tweets(custom_df, ngram_range=(1, 3), stop_words='english')

<ipython-input-7-4a6b54e8f354>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_tweet'] = df['tweet'].apply(clean_tweet)
<ipython-input-7-4a6b54e8f354>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokenized_tweet'] = df['cleaned_tweet'].apply(word_tokenize)
<ipython-input-7-4a6b54e8f354>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [39]:
preprocessed_test_data1

(<10x316 sparse matrix of type '<class 'numpy.float64'>'
 	with 323 stored elements in Compressed Sparse Row format>,
 array(['65', '65 private', '65 private jet', 'accountant',
        'accountant shes', 'accountant shes super', 'admit', 'admit think',
        'admit think georgia', 'animal', 'animal athlete',
        'animal athlete riding', 'apologise', 'apologise bhai',
        'apologise bhai thing', 'army', 'army run', 'army run fuck',
        'athlete', 'athlete riding', 'athlete riding bicycle', 'attracted',
        'attracted physically', 'attracted physically mean', 'avdeevka',
        'avdeevka village', 'avdeevka village lastochkino', 'bhai',
        'bhai thing', 'bhai thing parcel', 'bicycle', 'bicycle drone',
        'bicycle drone camera', 'bicycle race', 'bicycle race ocean',
        'blyatnsky', 'blyatnsky army', 'blyatnsky army run', 'camera',
        'camera view', 'chartered', 'chartered accountant',
        'chartered accountant shes', 'chelseas', 'chelseas insecu

In [40]:
custom_df

,tweet,labels_self,cleaned_tweet,tokenized_tweet,lemmatized_tweet
0,"Blyatnsky Army- “Run the fuck, why are you so ...",OFF,blyatnsky army run fuck fucking slow ukraine m...,"[blyatnsky, army, run, fuck, fucking, slow, uk...",blyatnsky army run fuck fucking slow ukraine m...
1,“you dont get to tell me about sad” well fuck,OFF,dont get tell sad well fuck,"[dont, get, tell, sad, well, fuck]",dont get tell sad well fuck
2,Chelsea’s insecurities drove her to put hersel...,OFF,chelseas insecurities drove put one sided comp...,"[chelseas, insecurities, drove, put, one, side...",chelseas insecurity drove put one sided compet...
3,Nah all these islanders are pussies they need ...,OFF,nah islanders pussies need admit think georgia...,"[nah, islanders, pussies, need, admit, think, ...",nah islander pussy need admit think georgia h ...
4,"Hello, I’m 6’5 and I have a private jet. I can...",NOT,hello im 65 private jet cook clean im studying...,"[hello, im, 65, private, jet, cook, clean, im,...",hello im 65 private jet cook clean im studying...
5,Guys are sending their n*des to me in the dms ...,OFF,guys sending ndes dms wtf,"[guys, sending, ndes, dms, wtf]",guy sending ndes dm wtf
6,"Met a fabulous entrepreneur for lunch, Rachana...",NOT,met fabulous entrepreneur lunch rachana ranade...,"[met, fabulous, entrepreneur, lunch, rachana, ...",met fabulous entrepreneur lunch rachana ranade...
7,A bicycle race on ocean with different animals...,NOT,bicycle race ocean different animals athletes ...,"[bicycle, race, ocean, different, animals, ath...",bicycle race ocean different animal athlete ri...
8,Only hot people can complete the whole syllabu...,NOT,hot people complete whole syllabus one night exam,"[hot, people, complete, whole, syllabus, one, ...",hot people complete whole syllabus one night exam
9,"You don't need to apologise @USER bhai, these ...",NOT,dont need apologise bhai things part parcel ga...,"[dont, need, apologise, bhai, things, part, pa...",dont need apologise bhai thing part parcel gam...


In [41]:
X_custom_tfidf = tfidf_vectorizer.fit_transform(custom_df["lemmatized_tweet"].ravel())

In [42]:
predictions_a = model_a.predict(X_custom_tfidf)

In [43]:
predictions_a
#1 means offensive, 0 means not offensive

array([1, 0, 1, 1, 1, 1, 0, 1, 0, 0])